In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from google.cloud import language_v1

from datetime import datetime
import re
from collections import defaultdict
import json
import pprint

from crawler_model import Ptt

ua = UserAgent()
fakeua = ua.random

In [ ]:
# Read
with open('phone_catalog.json', 'r') as fp:
    phone_dict = json.load(fp)

# phone_list

In [37]:
def get_phones():
    url = "https://en.wikipedia.org/wiki/List_of_Android_smartphones"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    tables = soup.find_all("table", class_="wikitable")
    rows = []
    for i in tables:
        rows.extend(i.select("tbody tr th"))

    phones = [a.text.partition("(")[0].strip() for a in rows]
    phones = [ p for p in phones if len(p.split()) > 1]

    brands = set()
    for p in phones:
        brands.add(p.split(" ")[0])
    brands = list(brands)
    brands = sorted(brands)
    for b in range(len(brands)):
        if brands[b] == "Asus":
            brands = brands[b:]
            break

    phone_list = []
    for b in brands:
        if b == "Samsung":
            for p in phones:
                if p.split(" ")[0] == b:
                    if "/" in p:
                        p_lst = p.split("/")
                        for series in p_lst[1:]:
                            sub_p = p_lst[0].replace("5G", "").strip()
                            if series == "+":
                                phone = sub_p + series
                            else:
                                phone = sub_p + " " + series
                            if len(phone) > 0:
                                phone_list.append(phone)
                        phone_list.append(sub_p)
                    else:
                        phone = p.replace("5G", "").strip()
                        if len(phone) > 0:
                            phone_list.append(phone)
        elif b == "POCO":
            for p in phones:
                if p.split(" ")[0] == b:
                    phone = p.replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_list.append(phone)

        elif b == "Pixel":
            for p in phones:
                if p.split(" ")[0] == b:
                    phone = p.replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_list.append(phone)
        else:
            for p in phones:
                if p.split(" ")[0] == b:
                    if "/" in p:
                        p_lst = p.split("/")
                        for series in p_lst[1:]:
                            sub_p = p_lst[0].replace("5G", "").strip()
                            if series == "+":
                                phone = sub_p + series
                            else:
                                phone = sub_p + " " + series
                            if len(phone) > 0:
                                phone_list.append(phone)
                        phone_list.append(sub_p)
                    else:
                        phone = p.replace("5G", "").strip()
                        if len(phone) > 0:
                            phone_list.append(phone)
                        phone_list.append(phone)
    phone_list.append("Asus ROG Phone 2")
    extend = ["Note 20 Ultra", "S20 Ultra", "S21 Ultra", "S20+", "S21+", "S20", "S21"]
    for s in extend:
        phone_list.append(s)

    url = "https://www.theiphonewiki.com/wiki/List_of_iPhones"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    table = soup.find("div", id="toc")
    iphones = []
    for span in table.select("ul li span"):
        if "iPhone" in str(span):
            if ("SE" in str(span)) and ("1" in str(span)):
                iphones.append("iPhone SE")
            elif ("SE" in str(span)) and ("2" in str(span)):
                iphones.append("iPhone SE2")
            else:
                iphones.append(span.text)
    iphones.remove("iPhone")
    phone_list.extend(iphones)
    phone_set = set()
    for p in phone_list:
        phone_set.add(p)
    phone_list = sorted(list(phone_set), key=lambda x: len(x), reverse=True)
    phone_list.remove("Asus ROG Phone II")
    return phone_list

# crawl PTT

In [ ]:
def get_last_page(url):
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    previous = soup.find_all("a", class_="btn wide")[0]["href"]
    previous_page_num = int(re.findall(r"\d+", previous)[0])
    return previous_page_num
last_page = get_last_page("https://www.ptt.cc/bbs/MobileComm/search?page=1&q=%E5%BF%83%E5%BE%97")
last_page

In [ ]:
def sample_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.path.join(os.path.expanduser("~"), "smartphone-smartprice-key.json")

    post = {}
    client = language_v1.LanguageServiceClient()

    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    document = {"content": text_content, "type_": type_}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    # Get overall sentiment of the input document
    doc = {}
    doc["score"] = response.document_sentiment.score
    
    doc["magnitude"] = response.document_sentiment.magnitude

    # Get sentiment for all sentences in the document
    sentences = []
    for sentence in response.sentences:
        # if abs(sentence.sentiment.score) >= 0.5 and sentence.sentiment.magnitude >= 0.5:
        d = {}
        d["content"] = sentence.text.content.replace(".", "").replace("。", "")
        d["score"] = sentence.sentiment.score
        d["magnitude"] = sentence.sentiment.magnitude
        sentences.append(d)
    
    return doc, sentences

def crawl_comm(links):
    for link in links:
        url = "https://www.ptt.cc" + link[1]
        # GET request from url and parse via BeautifulSoup
        ua = UserAgent()
        fakeua = ua.random
        headers = {"Origin": "https://www.ptt.cc",
                "Referer": "https://fonts.googleapis.com/",
                "sec-ch-ua-mobile": "?0",
                "User-Agent": fakeua}
        resp = requests.get(url, headers=headers)
        if "200" in str(resp):
            soup = BeautifulSoup(resp.text, "html.parser")
            main = soup.find("div", id="main-content")
            print(main.text)
            # keywords = ["優點", "缺點", "照相", "電池", "續航"]
            if "※ 引述" in str(main):   
                body = str(main).split('※ 引述')[0].split('</span></div>')[-1].replace(" ","").replace("\n", "。")
                body = BeautifulSoup(body).text
            else:
                body = str(main).split('--')[0].split('</span></div>')[-1].replace(" ","").replace("\n", "。")
                body = BeautifulSoup(body).text
        doc, sentences = sample_analyze_sentiment(body)
        d = {"title": link[0], "link": link[1], "doc": doc, "sentences": sentences}
        print(d)
        time.sleep(5)


In [ ]:
raw_data = []
data = []
error_links = []
for page in range(1, 2):

    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}

    url = "https://www.ptt.cc/bbs/MobileComm/search?page={}&q=%E5%BF%83%E5%BE%97".format(page)
    print(url)
    phone_list = get_phones()
    resp = requests.get(url, headers=headers)
    index_soup = BeautifulSoup(resp.text, "html.parser")
    keywords = ["換", "vs", "v.s", "vs.", "v.s."]
    title = index_soup.findAll("div", class_="title")

    links = []
    for t in title:
        if any(keyword in t.text for keyword in keywords):
            pass
        else:
            for p in phone_list:
                if p.replace(" ", "").lower() in t.text.replace(" ", "").lower():
                    print(p, t)
                    links.append( (t.text, "https://www.ptt.cc" + t.find("a")["href"]) )
                    break
                    break
        

    crawl_comm(links)

In [ ]:
print(links)

In [ ]:
import pprint
print(len(data))
data[0]

# Sentiment

In [ ]:
from sentifish import Sentiment
text='''2020/02自美國雅馬遜購入
昨天發現電池膨脹了
爬了一下板上資訊
應該可以更換保固區域後以延長保固1年為由
更換（整）新機

但也看到線上服務看得是一種運
不是大好就是大壞
不巧小弟抽到壞的那邊
客服堅持不是官網買的無法處理
也沒有換保固區域的處理
另外也說電池膨脹不在延長保固之範圍
（但電量消耗大是可以的）

總之就是叫我自己去找雅馬遜處理

我想這樣的客服
我大概不會再買pixel了吧'''
obj=Sentiment(text)
polarity = obj.analyze( )
polarity

# mobile01

In [ ]:
headers = {
    'user-agent': fakeua,
    'x-api-source': 'pc',
    'referer': "https://shopee.tw/product/354450050/8754002931"
    }
r = requests.get("https://shopee.tw/product/354450050/8754002931", headers=headers)
print(r)
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettfy())

In [ ]:
s = requests.Session()
headers = {
    'origin': 'https://www.mobile01.com',
    'referer': "https://www.mobile01.com/",
    'sec-ch-ua': "Not A;Brand;v=99, Chromium;v=90, Google Chrome;v=90",
    'user-agent': fakeua,
    }
s.headers = headers
r = s.get("https://www.mobile01.com/newslist.php?type=1&c=16&date=2021")
print(r)
soup = BeautifulSoup(r.text, "html.parser")
a = soup.find("a", class_="c-articleCard")
links = [ d["href"] for d in a ]

In [ ]:
s = requests.Session()
headers = {
    'origin': 'https://www.mobile01.com',
    'referer': "https://www.mobile01.com/",
    'user-agent': fakeua,
    }
s.headers = headers
r = s.get("https://www.mobile01.com/topicdetail.php?f=754&t=6385086")
print(r)
soup = BeautifulSoup(r.text, "html.parser")
# a = soup.find("a", class_="c-articleCard")
# links = [ d["href"] for d in a ]

In [ ]:
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
url = "https://www.mobile01.com/newslist.php?type=1&c=16&date=2021"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(5)
pageSource = driver.page_source
driver.close()

In [ ]:
soup = BeautifulSoup(pageSource, "html.parser")
a = soup.findAll("a", class_="c-articleCard")

In [41]:
links = []
keywords = ["換", "vs", "v.s", "vs.", "v.s.", "跳", "比", "較", "戰"]
extend = ["Note 20 Ultra", "S20 Ultra", "S21 Ultra", "S20+", "S21+", "S20", "S21"]
phone_list = get_phones()
for d in a:
    t = d.find("div", class_="l-articleCardDesc").text.strip()
    if any(keyword in t for keyword in keywords):
        pass
    else:
        for p in phone_list:
            if p.replace(" ", "").lower() in t.replace(" ", "").lower():
                if p in extend:
                    p = "Samsung Galaxy " + p
                print(p, t)
                links.append( (p, t, "https://www.mobile01.com/" + d["href"]) )
                break
                break 

Sony Xperia 10 III Sony Xperia 10 III 試用｜最高防水和大電量 讓你安心在手機上人與人連結
Asus ZenFone 8 ASUS ZenFone 8 / 8 Flip體驗｜翻轉依舊、也玩輕巧！
Oppo Reno5 OPPO Reno5 Z試用 顏值輕巧兼具的5G中階
POCO X3 Pro POCO X3 Pro試用/ 大電量跟有CP值就是賣點
POCO F3 POCO F3試用/ 萬元買到Snapdragon 870高性能
Realme GT realme GT體驗｜效能特化、最便宜S888旗艦！
Vivo X60 Pro vivo X60 Pro 評測｜攝錄「穩」中求趣、效能直逼旗艦！
Asus ROG Phone 5 ASUS ROG Phone 5 Ultimate體驗｜回歸絢爛設計、機型再進化！
Oppo Reno5 OPPO Reno 5 雙機體驗｜同門不同派 效能是最大差異！
Samsung Galaxy S21 Ultra Snapdragon 888版Galaxy S21 Ultra玩遊戲紀錄 不是每款都有意外
Samsung Galaxy S21 三星Galaxy S21/S20各種測試 拍照/螢幕/散熱差異一次看！
Samsung Galaxy S21 Ultra 三星Galaxy S21 Ultra揭曉：百倍變焦進化、重點是軟體應用！
Oppo A73 OPPO A73 5G試用：輕鬆入手價格 輕鬆拍好人像


In [42]:
links

[('Sony Xperia 10 III',
  'Sony Xperia 10 III 試用｜最高防水和大電量 讓你安心在手機上人與人連結',
  'https://www.mobile01.com/topicdetail.php?f=569&t=6374069'),
 ('Asus ZenFone 8',
  'ASUS ZenFone 8 / 8 Flip體驗｜翻轉依舊、也玩輕巧！',
  'https://www.mobile01.com/topicdetail.php?f=588&t=6363391'),
 ('Oppo Reno5',
  'OPPO Reno5 Z試用 顏值輕巧兼具的5G中階',
  'https://www.mobile01.com/topicdetail.php?f=689&t=6357976'),
 ('POCO X3 Pro',
  'POCO X3 Pro試用/ 大電量跟有CP值就是賣點',
  'https://www.mobile01.com/topicdetail.php?f=634&t=6352979'),
 ('POCO F3',
  'POCO F3試用/ 萬元買到Snapdragon 870高性能',
  'https://www.mobile01.com/topicdetail.php?f=634&t=6350896'),
 ('Realme GT',
  'realme GT體驗｜效能特化、最便宜S888旗艦！',
  'https://www.mobile01.com/topicdetail.php?f=789&t=6349965'),
 ('Vivo X60 Pro',
  'vivo X60 Pro 評測｜攝錄「穩」中求趣、效能直逼旗艦！',
  'https://www.mobile01.com/topicdetail.php?f=754&t=6334883'),
 ('Asus ROG Phone 5',
  'ASUS ROG Phone 5 Ultimate體驗｜回歸絢爛設計、機型再進化！',
  'https://www.mobile01.com/topicdetail.php?f=588&t=6321569'),
 ('Oppo Reno5',
  'OPPO Reno 5 雙機體驗｜同門

In [44]:
data = []
for link in links[:1]:
    url = link[2]
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    time.sleep(3)
    pageSource = driver.page_source
    soup = BeautifulSoup(pageSource, "html.parser")
    created_at = soup.find("span", class_="o-fNotes o-fSubMini").text
    created_at = datetime.strptime(created_at, '%Y-%m-%d %H:%M')
    div = soup.find("div", itemprop="articleBody")
    body = div.text
    doc, sentences = sample_analyze_sentiment(body.replace("\n", "。"))
    driver.close()
    d = {"title": link[0], "arc_title": link[1], "link": link[2], "doc": doc, "sentences": sentences, "body": body, "created_at": created_at}
    data.append(d)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/addie_tyc/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache
